# ITP ID 221 RT Analysis

In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils
import utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

In [2]:
## parameters cell
itp_id = 300

In [3]:
# Parameters
district = 3
itp_id = 221


In [4]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    >> filter(_.itp_id == itp_id)
    >> collect()
)

In [5]:
# airtable_organizations

In [6]:
analysis_date = dt.date(2022, 3, 30)

In [7]:
trips = pd.read_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{itp_id}_03_30.parquet')

In [8]:
stop_delay = gpd.read_parquet(f'{utils.GCS_FILE_PATH}stop_delay_views/{itp_id}_03_30.parquet')
stop_delay['arrival_time'] = stop_delay.arrival_time.map(lambda x: dt.datetime.fromisoformat(x))
stop_delay['actual_time'] = stop_delay.actual_time.map(lambda x: dt.datetime.fromisoformat(x))

In [9]:
%%capture
routelines = utils.get_routelines(itp_id, analysis_date)

In [10]:
%%capture
rt_day = rt_filter_map_plot.RtFilterMapper(trips, stop_delay, routelines)

In [11]:
from IPython.display import display, Markdown, Latex, HTML

In [12]:
import json

In [13]:
#export_papermill
agency_name = rt_day.calitp_agency_name
district = airtable_organizations.caltrans_district.iloc[0]
print(json.dumps({'agency_name': agency_name, 'district': district}))

{"agency_name": "Gold Country Stage", "district": "03 - Marysville"}


In [14]:
# Bus Speed Maps and Delay Analysis {agency_name}, District {district}

In [15]:
display(HTML(f'<title> Bus Speed Maps and Delay Analysis {agency_name}, District {district} </title>'))

In [16]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [17]:
## avoid papermill errors when map fails to render
m = None

In [18]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [19]:
m

In [20]:
%%capture
rt_day.set_filter(start_time='07:00', end_time='09:00')

In [21]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [22]:
m

In [23]:
try:
    metrics_df = (rt_day.rt_trips
         >> group_by(_.route_short_name, _.direction_id)
         >> summarize(speed_variance = _.mean_speed_mph.var(),
                     mean_speed_mph = _.mean_speed_mph.mean(),
                     min_hour = _.median_time.min().hour,
                     max_hour = _.median_time.max().hour)
         >> filter(_.min_hour <= 7, _.max_hour >= 15,
                   _.mean_speed_mph < _.mean_speed_mph.quantile(.6),
                   _.speed_variance)
         >> arrange(-_.speed_variance)
         >> head(20)
        )
    metrics_df
except Exception as e:
    print(e)
## check that route has all-day span...

In [24]:
def map_from_metrics(rt_day, metrics_df):
    try:
        route_name = metrics_df.iloc[1,0]
        direction_id = metrics_df.iloc[1,1]
    except Exception as e:
        print(e)
        return
    # route_name = '281'
    # direction_id = '1'
    rt_day.set_filter(start_time='07:00', end_time='09:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        am_map = rt_day.segment_speed_map(how='low_speeds')
        display(am_map)
    except Exception as e:
        print(e)
        pass
    
    rt_day.set_filter(start_time='11:00', end_time='14:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        mid_map = rt_day.segment_speed_map(how='low_speeds')
        display(mid_map)
    except Exception as e:
        print(e)
        pass

    rt_day.set_filter(start_time='15:00', end_time='19:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        pm_map = rt_day.segment_speed_map(how='low_speeds')
        display(pm_map)
    except Exception as e:
        print(e)
        pass

    return

In [25]:
try:
    map_from_metrics(rt_day, metrics_df)
except Exception as e:
    print(e)
    pass

'GeoDataFrame' object has no attribute 'trip_id'
'GeoDataFrame' object has no attribute 'trip_id'
'GeoDataFrame' object has no attribute 'trip_id'
